In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import holidays
import datetime
#import folium
%matplotlib inline

2 Cells below is relevant only when activating on cloud / Linux machine.
Since on Windows  google.colab not working


from google.colab import files
uploaded = files.upload()

import io
DataFrame = pd.read_csv(io.BytesIO(uploaded['Crimes_-_2020.csv']))
DataFrame

Update csv data bease:
https://drive.google.com/file/d/1s5lavM_C9aLsgdFzUmXOfbh_XvSxl-G4/view?usp=sharing

In [2]:
DataFrame = pd.read_csv("Crimes_-_2019_to_Present.csv")

In [3]:
DataFrame.columns

Index([u'Unnamed: 0', u'ID', u'Case Number', u'Date', u'Block', u'IUCR',
       u'Primary Type', u'Description', u'Location Description', u'Arrest',
       u'Domestic', u'Beat', u'District', u'Ward', u'Community Area',
       u'FBI Code', u'X Coordinate', u'Y Coordinate', u'Year', u'Updated On',
       u'Latitude', u'Longitude', u'Location'],
      dtype='object')

In [4]:
Sunrise = pd.read_csv('sunrise-chicago.csv')


In [5]:
# Add converted columns
Sunrise['dtsunset']=Sunrise.apply(lambda x: datetime.datetime.combine(pd.to_datetime(x['date']), pd.to_datetime(x['sunset']).time()), axis=1)
Sunrise['dtsunrise']=Sunrise.apply(lambda x: datetime.datetime.combine(pd.to_datetime(x['date']), pd.to_datetime(x['sunrise']).time()), axis=1)
Sunrise.set_index(['date'], inplace=True)

In [6]:
# a constant defining the time between sunset and darkness
delta_sunset = datetime.timedelta(0,3600)
#TODO: check exact delta, set another constant for sunrise time



In [7]:
# Accepts datetime in string, returns boolean if it's in night time
def is_dark(datecol) :
  d = pd.to_datetime(datecol)
  row = Sunrise.loc[d.strftime("%Y-%m-%d")]
  return d < row['dtsunrise'] or d > (row['dtsunset'] + delta_sunset)

# Add IsDark column to DataFrame
# For tests, uncomment the next line and comment the line after it
#DataFrame = DataFrame.head()
#DataFrame['IsDark']=DataFrame.apply(lambda x: is_dark(x['Date']) ,axis=1)

Based on link below, we shall use FBI code instead of IUCR .
http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html#N04A
For example :

> **FBI code** - Homicide 1st & 2nd Degree - 01A

> **IUCR** 
*  0110 HOMICIDE FIRST DEGREE MURDER
*  0130 HOMICIDE SECOND DEGREE MURDER

> **FBI code** - Nonvoluntary Manslaughter - 01B

> **IUCR** 
*    0141 HOMICIDE INVOLUNTARY MANSLAUGHTER
*    0142 HOMICIDE RECKLESS HOMICIDE

**dt0 - Seasons numeric**

*   4- Fall   - Lowest crime rate
*   3- Summer - Highest crime rate

*   2- Spring
*   1- Winter

**dt1 - Weekday numeric**
 The day of the week with Monday=0, Sunday=6 
 
**dt2 - Weekday string** 

*  Sunday - Highest rate crimes
*  Tuesday - Lowest rate crimes


   
    


In [8]:
dtTemp = DataFrame.copy()

In [9]:
dtTemp.drop(['ID', 'Case Number', 'Updated On', 'Block', 'IUCR', 'Description', 'Domestic','Arrest', 'Beat', 'Location', 'X Coordinate', 'Y Coordinate', 'Ward', 'FBI Code', 'Latitude', 'Longitude', 'Unnamed: 0', 'Year', 'Primary Type', 'District', 'Location Description'], 1, inplace=True)
dtTemp

,Date,Community Area
0,03/19/2019 04:30:00 PM,13.0
1,03/19/2019 09:00:00 AM,31.0
2,02/11/2019 05:50:00 PM,22.0
3,03/20/2019 07:00:00 PM,2.0
4,03/20/2019 01:00:00 AM,77.0
5,03/20/2019 01:30:00 PM,25.0
6,02/01/2019 01:00:00 AM,16.0
7,03/31/2019 01:00:00 AM,8.0
8,03/31/2019 11:00:00 AM,32.0
9,03/21/2019 12:00:00 PM,24.0


In [10]:
null_counts = dtTemp.isnull().sum()
null_counts[null_counts>0]

Series([], dtype: int64)

Now dataset is clean no NaN values

In [11]:
for date in holidays.UnitedStates(years=2020).items():print(str(date[0]))

2020-12-25
2020-07-03
2020-11-26
2020-07-04
2020-10-12
2020-05-25
2020-01-20
2020-11-11
2020-09-07
2020-01-01
2020-02-17


In [12]:
us_holidays = []
for date in holidays.UnitedStates(years=2019).items():
    us_holidays.append(str(date[0]))
for date in holidays.UnitedStates(years=2020).items():
    us_holidays.append(str(date[0]))
    
print(us_holidays)

['2019-07-04', '2019-02-18', '2019-12-25', '2019-11-28', '2019-05-27', '2019-01-01', '2019-09-02', '2019-10-14', '2019-01-21', '2019-11-11', '2020-12-25', '2020-07-03', '2020-11-26', '2020-07-04', '2020-10-12', '2020-05-25', '2020-01-20', '2020-11-11', '2020-09-07', '2020-01-01', '2020-02-17']


In [13]:
dt0 = dtTemp.copy()
#dt0["Arrest"] = dt0["Arrest"].astype(int)
#dt0["Domestic"] = dt0["Domestic"].astype(int)
#dt0["FBI Code"] = (DataFrame["FBI Code"]).apply(lambda x: int(x, 16))
dt0["Season"] = (pd.to_datetime(dt0['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.month%12 + 3)//3
dt0["Weekday"] = pd.to_datetime(dt0['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.dayofweek
dt0["Just_Date"] = (pd.to_datetime(dt0['Date'])).dt.date
dt0["Is_Holiday"] = [ 1 if str(val).split()[0] in us_holidays else 0 for val in dt0["Just_Date"]]
dt0["Day_of_Month"] = pd.to_datetime(dt0["Just_Date"]).dt.day
dt0["Month"] = pd.to_datetime(dt0["Just_Date"]).dt.month
dt0["IsDark"]= DataFrame.apply(lambda x: is_dark(x['Date']) ,axis=1)


In [14]:
dt0.columns

Index([u'Date', u'Community Area', u'Season', u'Weekday', u'Just_Date',
       u'Is_Holiday', u'Day_of_Month', u'Month', u'IsDark'],
      dtype='object')

In [15]:
dt0

,Date,Community Area,Season,Weekday,Just_Date,Is_Holiday,Day_of_Month,Month,IsDark
0,03/19/2019 04:30:00 PM,13.0,2,1,2019-03-19,0,19,3,False
1,03/19/2019 09:00:00 AM,31.0,2,1,2019-03-19,0,19,3,False
2,02/11/2019 05:50:00 PM,22.0,1,0,2019-02-11,0,11,2,False
3,03/20/2019 07:00:00 PM,2.0,2,2,2019-03-20,0,20,3,False
4,03/20/2019 01:00:00 AM,77.0,2,2,2019-03-20,0,20,3,True
5,03/20/2019 01:30:00 PM,25.0,2,2,2019-03-20,0,20,3,False
6,02/01/2019 01:00:00 AM,16.0,1,4,2019-02-01,0,1,2,True
7,03/31/2019 01:00:00 AM,8.0,2,6,2019-03-31,0,31,3,True
8,03/31/2019 11:00:00 AM,32.0,2,6,2019-03-31,0,31,3,False
9,03/21/2019 12:00:00 PM,24.0,2,3,2019-03-21,0,21,3,False


Creating a map from  "Community Area" to map coordinates

In [ ]:
community_area_dict = dict()

for i in range(1,78):
    area_df = DataFrame[DataFrame["Community Area"]==i]
    vc = area_df["Block"].value_counts()
    max_block = vc.head(1).index[0]
    block_df = area_df[area_df["Block"]==max_block]
    var = block_df["Location"].iloc[0]
    count = 1
    while pd.isnull(var):
        var = block_df["Location"].iloc[count]
        count+=1
    community_area_dict[i] = var

In [25]:
dt0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423270 entries, 276 to 428911
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      423270 non-null  int64  
 1   Date            423270 non-null  object 
 2   Arrest          423270 non-null  int32  
 3   Domestic        423270 non-null  int32  
 4   Beat            423270 non-null  int64  
 5   District        423270 non-null  float64
 6   Ward            423270 non-null  float64
 7   Community Area  423270 non-null  float64
 8   FBI Code        423270 non-null  int64  
 9   X Coordinate    423270 non-null  float64
 10  Y Coordinate    423270 non-null  float64
 11  Latitude        423270 non-null  float64
 12  Longitude       423270 non-null  float64
 13  Season          423270 non-null  int64  
 14  Weekday         423270 non-null  int64  
 15  Just_Date       423270 non-null  object 
 16  Is_Holiday      423270 non-null  int64  
 17  Day_of_M

In [26]:
ndf = dt0.groupby(['Is_Holiday', 'Day_of_Month', 'Weekday', 'Season', 'IsDark']).size().reset_index(name='Counts')
ndf

,Is_Holiday,Day_of_Month,Weekday,Season,IsDark,Counts
0,0,1,0,2,False,533
1,0,1,0,2,True,240
2,0,1,0,3,False,1137
3,0,1,0,3,True,606
4,0,1,1,4,False,928
...,...,...,...,...,...,...
1017,1,25,2,1,True,298
1018,1,27,0,2,False,494
1019,1,27,0,2,True,252
1020,1,28,3,4,False,235


In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

In [28]:
X = ndf.drop('Counts', axis = 1)
Y = ndf['Counts']

In [29]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.2)
classifier= DecisionTreeClassifier()
classifier.fit(x_train, y_train)

DecisionTreeClassifier()

In [30]:
y_pred = classifier.predict(x_test)

In [31]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         1
         132       0.00      0.00      0.00         0
         133       0.00      0.00      0.00         1
         139       0.00      0.00      0.00         0
         148       0.00      0.00      0.00         1
         150       0.00      0.00      0.00         1
         151       0.00      0.00      0.00         0
         156       0.00      0.00      0.00         0
         167       0.00      0.00      0.00         1
         168       0.00      0.00      0.00         0
         170       0.00      0.00      0.00         1
         174       0.00      0.00      0.00         0
         175       0.00      0.00      0.00         1
         179       0.00      0.00      0.00         3
         183       0.00      0.00      0.00         1
         184       0.00    

C:\Users\bdl011\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bdl011\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
Check_Holiday = dt0[dt0["Is_Holiday"]==1]
Check_Holiday

In [ ]:
dt0["Month"].value_counts()

Should we  remove rows related to 6 days at the beginning of September?

In [ ]:
x=np.c_[dt0["FBI Code"]]
y=np.c_[dt0["Day_of_Month"]]
dt0.plot(kind='scatter', x="FBI Code", y='Day_of_Month', alpha=(0.4),figsize=(30,5))
plt.show()

In [ ]:
x=np.c_[dt0["FBI Code"]]
y=np.c_[dt0["Date"]]
dt0.plot(kind='scatter', x="FBI Code", y='Date', alpha=(0.4),figsize=(30,5))
plt.show()

In [ ]:
codt1=dt1[dt1["Date"]==6]
codt1.head(3)

In [ ]:
codt1.plot(kind='scatter',x="X Coordinate", y="Y Coordinate", alpha=(0.4),figsize=(20,20),c="Date", cmap=plt.get_cmap("Blues"), colorbar=True, sharex=False)
plt.legend()

In [ ]:
dt2 = DataFrame.copy()
dt2['Date'] = pd.to_datetime(dt2['Date'], format="%m/%d/%Y %I:%M:%S %p").dt.day_name()
dt2

In [ ]:
dt2["Date"].value_counts()

In [ ]:
DataFrame.shape[0]

In [ ]:
DataFrame.shape[1]

In [ ]:
DataFrame["Community Area"].value_counts()

Community Area no. 25 appears 8722 time in the data set

In [ ]:
DataFrame.Description.value_counts()

In [ ]:
DataFrame["Location Description"].value_counts()

In [ ]:
DataFrame.sort_values(by=["Location Description", "Primary Type"])

In [ ]:
DataFrame[(DataFrame["Primary Type"]=="ASSAULT")&(DataFrame["Location Description"]=="STREET")]

In [ ]:
DataFrame.plot(kind='scatter',x="X Coordinate", y="Y Coordinate", alpha=(0.4),figsize=(50,50),c="Community Area", cmap=plt.get_cmap("jet"), colorbar=True, sharex=False)
plt.legend()

In [ ]:
co.plot(kind='scatter',x="X Coordinate", y="Y Coordinate", alpha=(0.4),figsize=(10,10),c="Community Area", cmap=plt.get_cmap("Blues"), colorbar=True, sharex=False)
plt.legend()

Plot histogram that shows per district the number of total crimes.
Next step would be to create a testing data frame , from 20% of the dataset.
Need to make sure that it would resemble  the distribution of all DataSet

29/10/20 START

In [ ]:
dt0.describe()

In [ ]:
corrMatrix= dt0.corr()
corrMatrix["FBI Code"].sort_values(ascending=True)

29/10/20 What is the most common crime  (6 is the most common crime which is theft)

In [ ]:
dt0["Community Area"].value_counts()

In [ ]:
dt0["FBI Code"].value_counts()

In [ ]:
for6=dt0[dt0["FBI Code"]==6]
for6

In [ ]:
for6["Community Area"].value_counts()

From above area 8 has the highest theft crime. I expected based on below linear regression that 36 area would be the one

In [ ]:
x=np.c_[dt0["FBI Code"]]
y=np.c_[dt0["Community Area"]]
dt0.plot(kind='scatter', x="FBI Code", y='Community Area', alpha=(0.4),figsize=(30,10))
plt.show()

In [ ]:
import sklearn.linear_model
model = sklearn.linear_model.LinearRegression()
model.fit(x, y)

In [ ]:
X_new = [[6]]

In [ ]:
print(model.predict(X_new))

In [ ]:
for36=dt0[dt0["Community Area"]==36]
for36

In [ ]:
for36["FBI Code"].value_counts()

In [ ]:
dt0["FBI Code"].value_counts()

In [ ]:
X_new = [[27]]

In [ ]:
print(model.predict(X_new))

29/10/20 END

In [ ]:
DataFrame["Community Area"].value_counts().plot(kind='bar',figsize=(20,5))

In [ ]:
from sklearn.model_selection import train_test_split
train_frame, test_frame = train_test_split(DataFrame)

In [ ]:
test_frame["Community Area"].value_counts().plot(kind='bar',figsize=(20,5))

In [ ]:
train_frame["Community Area"].value_counts().plot(kind='bar',figsize=(20,5))

In [ ]:
len(train_frame)

In [ ]:
len(test_frame)

Convert "FBI Code" Hexa string to numeric value

In [ ]:
dt1['FBI Code'] = (DataFrame["FBI Code"]).apply(lambda x: int(x, 16))
dt1

In [ ]:
from sklearn import linear_model
from sklearn.utils import shuffle

In [ ]:
dt0=dt0[["Date","Arrest","Domestic","Ward","Community Area","FBI Code"]]
dt0

In [ ]:
dt0.info()

In [ ]:
dt0[(dt0["Date"]=="NaN")]

In [ ]:
predict="FBI Code"
X=np.array(dt0.drop([predict],1))

In [ ]:
type(X)

In [ ]:
Y=np.array(dt0[predict])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.1)

In [ ]:
linear=linear_model.LinearRegression()

In [ ]:
dt0

In [ ]:
def clean_dataset(dt0):
    assert isinstance(dt0, pd.DataFrame), "df needs to be a pd.DataFrame"
    dt0.dropna(inplace=True)
    indices_to_keep = ~dt0.isin([np.nan, np.inf, -np.inf]).any(1)
    return dt0[indices_to_keep].astype(np.float64)

In [ ]:
dt0.dropna()